In [1]:
! pip install rapidfuzz


In [2]:
from rapidfuzz import process, fuzz
import pandas as pd
import logging
import argparse
import pyodbc
import sys
import os
import re


In [3]:
! pip install mysql-connector

In [4]:
import mysql.connector

df1 = pd.read_excel(r'D:\KTs\python_work\Investor.xlsx')
df1.columns

Index(['Fund ID', 'Fund', 'MCPID', 'Investor Name',
       'Call: Investments|CAPCALL'],
      dtype='object')

In [5]:
s=df1[['MCPID', 'Investor Name']]

In [6]:
#result_list = [[r[0], r[1]] for r in s]

In [7]:
#result_list

In [8]:
df2 = pd.read_csv(r'D:\KTs\python_work\account.csv')
df2

,account,current_transaction,entry_date,amount,currency,debit_credit,description,type,transaction_description_61,description_nextline_61,txn_description_86,full description
0,210663800,230217DD1500000,02/17/2023,-1500000.00,USD,D,NLDP0//217122353,":61:230217DD1500000,00NLDP0//217122353",NLDP0//217122353,NaN,481/LOAN PAYMENT/LOAN AFS PAYMENT 279914-43514...,481/LOAN PAYMENT/LOAN AFS PAYMENT 279914-435...
1,210663800,230217DD1700000,02/17/2023,-1700000.00,USD,D,NLDP0//217121843,":61:230217DD1700000,00NLDP0//217121843",NLDP0//217121843,NaN,481/LOAN PAYMENT/LOAN AFS PAYMENT 279914-43513...,481/LOAN PAYMENT/LOAN AFS PAYMENT 279914-435...
2,210663509,230217CD1845,02/17/2023,1845.61,USD,C,NTRF230217049320//230217049320,":61:230217CD1845,61NTRF230217049320//230217049320",NTRF230217049320//230217049320,NaN,195/INCOMING WIRE DOM/0217MMQFMP31004356021717...,195/INCOMING WIRE DOM/0217MMQFMP310043560217...
3,210663509,230217DD372300,02/17/2023,-372300.82,USD,D,NTRF5238551,":61:230217DD372300,82NTRF5238551",NTRF5238551,NaN,495/BSuite Wire Out DOM/0217L2LFCK1C000910///B...,495/BSuite Wire Out DOM/0217L2LFCK1C000910//...
4,210766308,230217DD3200000,02/17/2023,-3200000.00,USD,D,NLDP0//217131828,":61:230217DD3200000,00NLDP0//217131828",NLDP0//217131828,NaN,481/LOAN PAYMENT/LOAN AFS PAYMENT 315648-65145...,481/LOAN PAYMENT/LOAN AFS PAYMENT 315648-651...
...,...,...,...,...,...,...,...,...,...,...,...,...
657,210766308,230309CD200000,03-09-2023,200000.00,USD,C,NLDP0//50000309102919,":61:230309CD200000,00NLDP0//50000309102919",NLDP0//50000309102919,NaN,171/TRANSFER FROM LOAN/LN 315648-001171/TRANSF...,171/TRANSFER FROM LOAN/LN 315648-001171/TRAN...
658,210663509,230309CD533,03-09-2023,533.01,USD,C,NTRF230309040717//230309040717,":61:230309CD533,01NTRF230309040717//230309040717",NTRF230309040717//230309040717,NaN,195/INCOMING WIRE DOM/0309L3LF151C003919030916...,195/INCOMING WIRE DOM/0309L3LF151C0039190309...
659,210663509,230309CD200000,03-09-2023,200000.00,USD,C,NLDP0//50000309102721,":61:230309CD200000,00NLDP0//50000309102721",NLDP0//50000309102721,NaN,171/TRANSFER FROM LOAN/LN 279842-001171/TRANSF...,171/TRANSFER FROM LOAN/LN 279842-001171/TRAN...
660,210663509,230309DD167540,03-09-2023,-167540.81,USD,D,NTRF5396226,":61:230309DD167540,81NTRF5396226",NTRF5396226,NaN,495/BSuite Book Trans DR////Reimbursement of e...,495/BSuite Book Trans DR////Reimbursement of...


In [9]:
#df_merged=df1+s
#df_merged = df1.append(df2, ignore_index=True)
#df_merged = pd.concat([df1, df2])
#df_merged


In [10]:
#df_merged.columns

In [11]:
grouped_file_2 = df2.groupby("full description")["account"].apply(set).reset_index()

In [12]:
for name, mcpid in zip(df1["Investor Name"], df1["MCPID"]):
        best_score = 0
        for desc in df2["full description"]:
            score = fuzz.token_set_ratio(name, desc)  
            if score > best_score:
                best_score = score
                match = grouped_file_2[grouped_file_2["full description"] == desc].iloc[0]
        accounts = match["account"]
        for account in accounts:
            mask = (df2["full description"] == match["full description"]) & (df2["account"] == account)
            df2.loc[mask, "Investor Name"] = name
            df2.loc[mask, "MCPID"] = mcpid
        # print(f"Best score for {name}: {best_score}")

In [13]:
#df_merged=df_merged.reset_index(drop=True)

In [14]:
#df_merged

In [15]:
df2.columns

Index(['account', 'current_transaction', 'entry_date', 'amount', 'currency',
       'debit_credit', 'description', 'type', 'transaction_description_61',
       'description_nextline_61', 'txn_description_86', 'full description',
       'Investor Name', 'MCPID'],
      dtype='object')

In [16]:
df2

,account,current_transaction,entry_date,amount,currency,debit_credit,description,type,transaction_description_61,description_nextline_61,txn_description_86,full description,Investor Name,MCPID
0,210663800,230217DD1500000,02/17/2023,-1500000.00,USD,D,NLDP0//217122353,":61:230217DD1500000,00NLDP0//217122353",NLDP0//217122353,NaN,481/LOAN PAYMENT/LOAN AFS PAYMENT 279914-43514...,481/LOAN PAYMENT/LOAN AFS PAYMENT 279914-435...,NaN,NaN
1,210663800,230217DD1700000,02/17/2023,-1700000.00,USD,D,NLDP0//217121843,":61:230217DD1700000,00NLDP0//217121843",NLDP0//217121843,NaN,481/LOAN PAYMENT/LOAN AFS PAYMENT 279914-43513...,481/LOAN PAYMENT/LOAN AFS PAYMENT 279914-435...,NaN,NaN
2,210663509,230217CD1845,02/17/2023,1845.61,USD,C,NTRF230217049320//230217049320,":61:230217CD1845,61NTRF230217049320//230217049320",NTRF230217049320//230217049320,NaN,195/INCOMING WIRE DOM/0217MMQFMP31004356021717...,195/INCOMING WIRE DOM/0217MMQFMP310043560217...,NaN,NaN
3,210663509,230217DD372300,02/17/2023,-372300.82,USD,D,NTRF5238551,":61:230217DD372300,82NTRF5238551",NTRF5238551,NaN,495/BSuite Wire Out DOM/0217L2LFCK1C000910///B...,495/BSuite Wire Out DOM/0217L2LFCK1C000910//...,NaN,NaN
4,210766308,230217DD3200000,02/17/2023,-3200000.00,USD,D,NLDP0//217131828,":61:230217DD3200000,00NLDP0//217131828",NLDP0//217131828,NaN,481/LOAN PAYMENT/LOAN AFS PAYMENT 315648-65145...,481/LOAN PAYMENT/LOAN AFS PAYMENT 315648-651...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,210766308,230309CD200000,03-09-2023,200000.00,USD,C,NLDP0//50000309102919,":61:230309CD200000,00NLDP0//50000309102919",NLDP0//50000309102919,NaN,171/TRANSFER FROM LOAN/LN 315648-001171/TRANSF...,171/TRANSFER FROM LOAN/LN 315648-001171/TRAN...,NaN,NaN
658,210663509,230309CD533,03-09-2023,533.01,USD,C,NTRF230309040717//230309040717,":61:230309CD533,01NTRF230309040717//230309040717",NTRF230309040717//230309040717,NaN,195/INCOMING WIRE DOM/0309L3LF151C003919030916...,195/INCOMING WIRE DOM/0309L3LF151C0039190309...,NaN,NaN
659,210663509,230309CD200000,03-09-2023,200000.00,USD,C,NLDP0//50000309102721,":61:230309CD200000,00NLDP0//50000309102721",NLDP0//50000309102721,NaN,171/TRANSFER FROM LOAN/LN 279842-001171/TRANSF...,171/TRANSFER FROM LOAN/LN 279842-001171/TRAN...,NaN,NaN
660,210663509,230309DD167540,03-09-2023,-167540.81,USD,D,NTRF5396226,":61:230309DD167540,81NTRF5396226",NTRF5396226,NaN,495/BSuite Book Trans DR////Reimbursement of e...,495/BSuite Book Trans DR////Reimbursement of...,NaN,NaN


In [17]:
df2.to_excel(r'D:\KTs\python_work\New Microsoft Excel Worksheet.xlsx', index=False)